# Time Series Feature: Time Series at Zurich Airport

This notebook demonstrates efficient geolocation and time-series access using Polytope feature extraction. By retrieving only the requested grid points, the I/O is greatly reduced and workflows can be considerably sped up.

<div style="text-align:center;">
  <img src="./images/t_2m_time_series.png" style="width:50%;"/>
</div>

## Installation
Follow the instructions in [README.md](https://github.com/MeteoSwiss/nwp-fdb-polytope-demo/blob/main/README.md#Installation-1) to install the necessary dependencies.

## Configuring access to Polytope
To access ICON data via MeteoSwiss's Polytope, you need a Polytope offline token provided by MeteoSwiss. If you do not already have a token, you can request one [here](https://meteoswiss.atlassian.net/wiki/spaces/IW2/pages/327780397/Polytope#Offline-token-authentication).

In [1]:
import os

# ECMWF Polytope (leave blank if using ICON-CSCS)
os.environ["POLYTOPE_USER_KEY"] = ""
os.environ["POLYTOPE_USER_EMAIL"] = ""

# ICON-CSCS Polytope (leave blank if using ECMWF)
os.environ["POLYTOPE_USERNAME"] = "xxx"
os.environ["POLYTOPE_PASSWORD"] = "xxx"
os.environ["POLYTOPE_ADDRESS"] = "https://polytope-dev.mchml.cscs.ch"

## Select the geolocation
We’ll request a time series at **Zurich Airport** (WGS84 coordinates).

In [2]:
zrh_geo_point = (8.565074, 47.453928)  # (longitude, latitude) in WGS84

## Rotate the point

Given that the data source accessed by Polytope is stored on a rotated grid, it is necessary to provide Polytope with the point in rotated coordinates, using a South Pole rotation with a reference of longitude 10° and latitude of -43°.

> **NOTE:** `transform_point()` expects `(longitude, latitude)`.

In [3]:
import cartopy.crs as ccrs

geo_crs = ccrs.PlateCarree()
rotated_crs = ccrs.RotatedPole(pole_longitude=190, pole_latitude=43)

# Convert the point from geographic to rotated coordinates
geo_lon, geo_lat = zrh_geo_point
rot_lon, rot_lat = rotated_crs.transform_point(geo_lon, geo_lat, geo_crs)
rotated_point = (rot_lon, rot_lat)

## Select the run date and time
The FDB containing real-time data typically **holds only the most recent day of forecasts**. The next cell computes a valid run time automatically.

In [4]:
from datetime import datetime, timedelta

now = datetime.now()
past_time = now - timedelta(hours=12)                # use a recent past cycle
rounded_hour = (past_time.hour // 6) * 6             # round down to 6-hour cycles
rounded_time = past_time.replace(hour=rounded_hour, minute=0, second=0, microsecond=0)

date = rounded_time.strftime('%Y%m%d')
time = rounded_time.strftime('%H%M')
date, time

('20250916', '1800')

## Define the feature
Request a **time series at one point** (`rotated_point`) over **steps 0–120**. ICON-CH2-EPS is hourly in this range (use 0–33 for ICON-CH1-EPS).

> **NOTE:** If points are `(longitude, latitude)`, set `axes=["longitude", "latitude"]`.

In [5]:
feature = {
    "type": "timeseries",
    "points": [rotated_point],      # single point in the rotated grid (lon, lat)
    "time_axis": "step",
    "range": {"start": 0, "end": 120},   # hourly steps 0..120
    "axes": ["longitude", "latitude"]    # order must match the point tuple
}

## Define the request
Use [meteodata-lab](https://meteoswiss.github.io/meteodata-lab/)'s `mars.Request` for a clean, validated API. This example fetches **2-m temperature** from **ICON-CH2-EPS** at the **surface**, for **ensemble member 1** (`number=1`), at the selected run date/time.
- `type="pf"` — perturbed member (requires `number`).
- `type="cf"` — control forecast (no `number`).

In [6]:
from meteodatalab import mars

request = mars.Request(
    param="T_2M",
    date=date,
    time=time,
    model=mars.Model.ICON_CH2_EPS,
    levtype=mars.LevType.SURFACE,
    type="pf",
    number=1,
    feature=feature
)

## Data retrieval
Load the data with [earthkit.data](https://earthkit-data.readthedocs.io/en/latest/) and convert it to an [xarray.Dataset](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.html).

In [7]:
import earthkit.data as ekd
ds = ekd.from_source(
    "polytope",
    "mchgj",
    request.to_polytope(),
    stream=False
).to_xarray()

2025-09-17 09:32:15 - INFO - Sending request...
{'request': 'class: od\n'
            "date: '20250916'\n"
            "expver: '0001'\n"
            'feature:\n'
            '  axes:\n'
            '  - longitude\n'
            '  - latitude\n'
            '  points:\n'
            '  - - -0.9702489359191186\n'
            '    - 0.4628136497221331\n'
            '  range:\n'
            '    end: 120\n'
            '    start: 0\n'
            '  time_axis: step\n'
            '  type: timeseries\n'
            'levtype: sfc\n'
            'model: icon-ch2-eps\n'
            'number: 1\n'
            "param: '500011'\n"
            'stream: enfo\n'
            "time: '1800'\n"
            'type: pf\n',
 'verb': 'retrieve'}
2025-09-17 09:32:15 - INFO - Request accepted. Please poll ./015edec2-cdcd-48d1-b7b8-570e4acc31a1 for status
2025-09-17 09:32:15 - INFO - Checking request status (015edec2-cdcd-48d1-b7b8-570e4acc31a1)...
2025-09-17 09:32:16 - INFO - The current status of the reques

## Plot the results

> **Tip:** Hover over the chart to see the exact timestamp and value (°C).  
> If you’re viewing a static image (e.g., PNG export), tooltips aren’t available.

In [8]:
from earthkit.plots.interactive import Chart


da = ds["t_2m"] - 273.15
da.name = "t_2m (°C)"

chart = Chart()
chart.line(da)

chart.show()

We can also query the entire ensemble at the same geolocation.

For ICON-CH2-EPS:

In [ ]:
import xarray as xr
import dataclasses as dc

ds_mems = []
for num in range(1, 21):
    req = dc.replace(
        request,
        number=str(num),
        model=mars.Model.ICON_CH2_EPS,
        feature={
            "type": "timeseries",
            "points": [rotated_point],
            "time_axis": "step",
            "range": {"start": 0, "end": 120},
            "axes": ["longitude", "latitude"],
        },
    )
    ds_mems.append(
        ekd.from_source("polytope", "mchgj", req.to_polytope(), stream=False).to_xarray()
    )

ds_ch2 = xr.concat(ds_mems, dim="number")

For ICON-CH1-EPS:

In [ ]:
import xarray as xr
ds_mems = []
for num in range(1, 11):
    req = dc.replace(
        request,
        number=str(num),
        model=mars.Model.ICON_CH1_EPS,
        feature={
            "type": "timeseries",
            "points": [rotated_point],
            "time_axis": "step",
            "range": {"start": 0, "end": 33},
            "axes": ["longitude", "latitude"],
        },
    )
    ds_mems.append(
        ekd.from_source("polytope", "mchgj", req.to_polytope(), stream=False).to_xarray()
    )

ds_ch1 = xr.concat(ds_mems, dim="number")

We can also request the same data from **IFS** using **ECMWF Polytope**. See the ECMWF Polytope client README for account creation: https://github.com/ecmwf/polytope-client/?tab=readme-ov-file#2-account-creation

In [11]:
import os

# Use your ECMWF Polytope credentials (do not hard-code secrets in notebooks)
os.environ["POLYTOPE_USER_KEY"] = "xxx"
os.environ["POLYTOPE_USER_EMAIL"] = "xxx"
os.environ["POLYTOPE_ADDRESS"] = "polytope.ecmwf.int"

> **Note:** For ECMWF Polytope, coordinates are **geographic** (WGS84). In the next request we pass `geo_lon` and `geo_lat`.

In [ ]:
request = {
    "class": "od",
    "stream": "enfo",
    "type": "pf",
    "date": date,
    "time": time,
    "levtype": "sfc",
    "expver": "0001",
    "domain": "g",
    "param": "167",
    "number": "1/to/50",
    "step": "0/to/160",
    "feature": {
        "type": "timeseries",
        "points": [[float(round(geo_lon, 2)), float(round(geo_lat, 2))]],
        "time_axis": "step",
        "axes": ["longitude", "latitude"]
    }
}

ds_ifs = ekd.from_source(
    "polytope",
    "ecmwf-mars",
    request,
    stream=False,
    address="polytope.ecmwf.int",
)

**Plotting ensemble members of time series**

In [14]:
from earthkit.plots.interactive import Chart

TIME_FREQUENCY = "1h"
QUANTILES = [0, 0.1, 0.25, 0.5, 0.75, 0.9, 1]

chart = Chart()

# CH1
chart.box(ds_ch1, quantiles=QUANTILES, line_color="#ea5545",
          legendgroup="CH1", showlegend=False)
chart.line(ds_ch1, aggregation="mean", line_color="grey",
           time_frequency=TIME_FREQUENCY,
           legendgroup="CH1", name="ICON-CH1-EPS (mean)", showlegend=True)

# CH2
chart.box(ds_ch2, quantiles=QUANTILES, line_color="#ef9b20",
          legendgroup="CH2", showlegend=False)
chart.line(ds_ch2, aggregation="mean", line_color="grey",
           time_frequency=TIME_FREQUENCY,
           legendgroup="CH2", name="ICON-CH2-EPS (mean)", showlegend=True)

# IFS
chart.box(ds_ifs, quantiles=QUANTILES, line_color="#27aeef",
          legendgroup="IFS", showlegend=False)
chart.line(ds_ifs, aggregation="mean", line_color="grey",
           time_frequency=TIME_FREQUENCY,
           legendgroup="IFS", name="IFS (mean)", showlegend=True)

chart.show()
